# G2Engine Reference

**!! If you wish to run any of Senzing's Java notebooks navigate to "Help" -> "Launch Classic Notebook" !!**

## Prepare Environment

In [1]:
import com.senzing.g2.engine.G2Engine;
import com.senzing.g2.engine.G2JNI;
import com.senzing.g2.engine.G2ConfigMgr;
import com.senzing.g2.engine.G2ConfigMgrJNI;
import com.senzing.g2.engine.Result;

### Helper class for Json Rendering

In [2]:
%%loadFromPOM
<dependency>
    <groupId>org.glassfish</groupId>
    <artifactId>javax.json</artifactId>
    <version>1.1.4</version>
</dependency>

In [3]:
import javax.json.*;
import static java.util.Collections.*;
import static javax.json.stream.JsonGenerator.PRETTY_PRINTING;

In [4]:
public class JsonUtil {
    private static final JsonWriterFactory PRETTY_FACTORY
        = Json.createWriterFactory(singletonMap(PRETTY_PRINTING, true));
        
    private static final JsonWriterFactory UGLY_FACTORY
        = Json.createWriterFactory(emptyMap());
    
    public static String toJsonText(JsonValue val) {
        return toJsonText(val, true);
    }

    public static String toJsonText(JsonValue val, boolean prettyPrint) {
        JsonWriterFactory factory = (prettyPrint) ? PRETTY_FACTORY : UGLY_FACTORY;
        StringWriter sw = new StringWriter();
        JsonWriter writer = factory.createWriter(sw);
        writer.write(val);
        sw.flush();
        return sw.toString();
    }
    
    public static JsonObject parseJsonObject(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readObject();
  }

    public static JsonArray parseJsonArray(String jsonText) {
        if (jsonText == null) return null;
        StringReader sr = new StringReader(jsonText);
        JsonReader jsonReader = Json.createReader(sr);
        return jsonReader.readArray();
  }
    
    
}

In [5]:
import java.util.UUID;
public static void RenderJSON(Object obj){
    String str = obj.toString();
    JsonObject json = JsonUtil.parseJsonObject(str);
    String Config = JsonUtil.toJsonText(json, false);
    UUID id = UUID.randomUUID();
    String uuid = id.toString();
    String div = "<div id=\""+ uuid +"\" style=\"height:100%; width:100%; background-color: LightCyan\"></div>";
    display(div, "text/html");
    String jav = "require([\"https://rawgit.com/caldwell/renderjson/master/renderjson.js\"], function() {document.getElementById(\'"+ uuid +"\').appendChild(renderjson("+json+"))});";
    display(jav, "application/javascript");
}

### Initialize Senzing configuration

Using environment variables and default values, create `senzingConfigJson`.
This value is used when instantiating Senzing objects.

In [6]:
// Get variables used in constructing Senzing Engine configuration.

String configPath = System.getenv("SENZING_ETC_DIR");
if (configPath == null) {
    configPath = "/etc/opt/senzing";
}

String supportPath = System.getenv("SENZING_DATA_VERSION_DIR");
if (supportPath == null) {
    supportPath = "/opt/senzing/data";
}

String g2Path = System.getenv("SENZING_G2_DIR");
if (g2Path == null) {
    g2Path = "/opt/senzing/g2";
}

String resourcePath = g2Path + "/resources";

String sqlConnection = System.getenv("SENZING_SQL_CONNECTION");
if (sqlConnection == null) {
    sqlConnection = "sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db";
}

String senzingLicenseBase64 = System.getenv("SENZING_LICENSE_BASE64_ENCODED");

// Construct the JSON string used for Senzing Engine configuration.
String senzingConfigJson;
if(senzingLicenseBase64!=null){
    senzingConfigJson = "{"
   + "\"PIPELINE\": {"
   +     "\"CONFIGPATH\": \"" + configPath + "\","
   +     "\"LICENSESTRINGBASE64\": \"" + senzingLicenseBase64 + "\","
   +     "\"SUPPORTPATH\": \"" + supportPath + "\","
   +     "\"RESOURCEPATH\": \"" + resourcePath + "\""
   + "},"
   + "\"SQL\": {"
   +     "\"CONNECTION\": \"" + sqlConnection + "\""
   + "}}";
}

else{
    senzingConfigJson = "{"
   + "\"PIPELINE\": {"
   +     "\"CONFIGPATH\": \"" + configPath + "\","
   +     "\"SUPPORTPATH\": \"" + supportPath + "\","
   +     "\"RESOURCEPATH\": \"" + resourcePath + "\""
   + "},"
   + "\"SQL\": {"
   +     "\"CONNECTION\": \"" + sqlConnection + "\""
   + "}}";
   
}
RenderJSON(senzingConfigJson);

<div id="a8875954-48f1-4b0f-b5d9-afbe824ebfc0" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('a8875954-48f1-4b0f-b5d9-afbe824ebfc0').appendChild(renderjson({"PIPELINE":{"CONFIGPATH":"/etc/opt/senzing","SUPPORTPATH":"/opt/senzing/data","RESOURCEPATH":"/opt/senzing/g2/resources"},"SQL":{"CONNECTION":"sqlite3://na:na@/var/opt/senzing/sqlite/G2C.db"}}))});

## G2Engine

### G2Engine initialization

To start using Senzing G2Engine, create and initialize an instance.
This should be done once per process.
###### Parameters

- **moduleName:** (str) A short name given to this instance of the G2Engine
  object.
- **senzingConfigJson:** (str) A JSON string containing configuration parameters.
- **verboseLogging:** (bool) A boolean which enables diagnostic logging.

Calling this function will return "0" upon success.

In [7]:
G2Engine g2engine = new G2JNI();

String moduleName = "ExampleG2Engine";
boolean verboseLogging = true;
int return_code = g2engine.init(moduleName, senzingConfigJson, verboseLogging);
System.out.print(return_code);

0

#### destroy()
destroy the engine so that we can initalize differently

In [8]:
g2engine.destroy();

0

### initWithConfigID

Alternatively `initWithConfigID()` can be used to specify a configuration.

###### Parameters

- **moduleName:** (str) A short name given to this instance of the G2Engine
  object.
- **senzingConfigJson:** (str) A JSON string containing configuration parameters.
- **verboseLogging:** (bool) A boolean which enables diagnostic logging.

Create G2Config so that we can use a configID

In [9]:
String moduleName = "ExampleG2Config";
boolean verboseLogging = true;

G2ConfigMgr g2configMgr = new G2ConfigMgrJNI();
int return_code = g2configMgr.init(moduleName, senzingConfigJson, verboseLogging);
Result<Long> config = new Result<Long>();
g2configMgr.getDefaultConfigID(config);
if(return_code!=0)
    System.out.print(g2configMgr.getLastException());
else
    System.out.print(return_code);

0

In [10]:
String moduleName = "ExampleG2Engine";
boolean verboseLogging = true;
int return_code = g2engine.initWithConfigID(moduleName, senzingConfigJson, config.getValue(), verboseLogging);

System.out.print(return_code);

0

### reinit

The `reinit()` function may be used to reinitialize the engine with a configuration

In [11]:
String moduleName = "ExampleG2Engine";
boolean verboseLogging = true;
int return_code = g2engine.reinit(config.getValue());

System.out.print(return_code);

0

### primeEngine

The `primeEngine()` method may optionally be called to pre-initialize
some of the heavier weight internal resources of the G2 engine.

In [12]:
int return_code= g2engine.primeEngine();
if(return_code!=0)
    System.out.print(g2engine.getLastException());
System.out.print(return_code)

0

### getActiveConfigID

Call `getActiveConfigID()` to return an identifier for the loaded
Senzing engine configuration.
The call will assign a long integer to a user-designated variable

###### Parameters

- **configID:** (long) The identifier value for the engine configuration. The result of function call is returned here

In [13]:
Result<Long> configID = new Result<Long>();
g2engine.getActiveConfigID(configID);

System.out.print(configID.getValue());

2425495991

### exportConfig

Call `exportConfig()` to retrieve your Senzing engine's configuration.
The call will assign a JSON document to a user-designated buffer,
containing all relevant configuration information
-- the function itself will return "0" upon success.
The exportConfig function accepts the following parameters as input:

###### Parameters

- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.
- **configID:** (long) The identifier value for the engine configuration. The result of function call is returned here

In [14]:
StringBuffer response = new StringBuffer();

g2engine.exportConfig(response, configID);
RenderJSON(response)

<div id="378ec009-b89d-4f46-85c0-d26cf2e5fc1d" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('378ec009-b89d-4f46-85c0-d26cf2e5fc1d').appendChild(renderjson({"G2_CONFIG":{"CFG_ETYPE":[{"ETYPE_ID":1,"ETYPE_CODE":"TEST","ETYPE_DESC":"Test","ECLASS_ID":1},{"ETYPE_ID":3,"ETYPE_CODE":"GENERIC","ETYPE_DESC":"Generic entity","ECLASS_ID":1}],"CFG_DSRC_INTEREST":[],"CFG_ECLASS":[{"ECLASS_ID":1,"ECLASS_CODE":"ACTOR","ECLASS_DESC":"Actor","RESOLVE":"Yes"}],"CFG_RCLASS":[{"RCLASS_ID":1,"RCLASS_CODE":"DERIVED","RCLASS_DESC":"Derived","IS_DISCLOSED":"No"},{"RCLASS_ID":2,"RCLASS_CODE":"DISCLOSED","RCLASS_DESC":"Disclosed","IS_DISCLOSED":"Yes"}],"CFG_FTYPE":[{"FTYPE_ID":1,"FTYPE_CODE":"NAME","FCLASS_ID":1,"FTYPE_FREQ":"NAME","FTYPE_EXCL":"No","FTYPE_STAB":"No","PERSIST_HISTORY":"Yes","USED_FOR_CAND":"No","DERIVED":"No","RTYPE_ID":0,"ANONYMIZE":"No","VERSION":2,"SHOW_IN_MATCH_KEY":"Yes"},{"FTYPE_ID":2,"FTYPE_CODE":"DOB","FCLASS_ID":2,"FTYPE_FREQ":"FM","FTYPE_EXCL":"Yes","FTYPE_STAB":"Yes

### stats

Call `stats()` to retrieve workload statistics for the current process.
These statistics will automatically reset after retrieval.

- **respone:** (String) Object to store the output of the method. can be used to print or store for later

In [15]:
String response = g2engine.stats();
RenderJSON(response);

<div id="c8fabe2d-c562-4a9d-a35e-032679b6b47e" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('c8fabe2d-c562-4a9d-a35e-032679b6b47e').appendChild(renderjson({"workload":{"loadedRecords":-1,"addedRecords":0,"deletedRecords":0,"reevaluations":0,"repairedEntities":0,"duration":0,"retries":0,"candidates":0,"actualAmbiguousTest":0,"cachedAmbiguousTest":0,"libFeatCacheHit":0,"libFeatCacheMiss":0,"unresolveTest":0,"abortedUnresolve":0,"gnrScorersUsed":1,"unresolveTriggers":{"normalResolve":0,"update":0,"relLink":0,"extensiveResolve":0,"ambiguousNoResolve":0,"ambiguousMultiResolve":0},"reresolveTriggers":{"abortRetry":0,"unresolveMovement":0,"multipleResolvableCandidates":0,"resolveNewFeatures":0,"newFeatureFTypes":[]},"reresolveSkipped":0,"filteredObsFeat":0,"expressedFeatureCalls":[],"expressedFeaturesCreated":[],"scoredPairs":[],"cacheHit":[],"cacheMiss":[],"redoTriggers":[],"latchContention":[],"highContentionFeat":[],"highContentionResEnt":[],"genericDetect":[],"candidateBu

### getRepositoryLastModifiedTime

Call `getRepositoryLastModifiedTime()` to obtain the last modified time of
the Senzing repository,measured in the number of seconds between the last
modified time and January 1, 1970 12:00am GMT (epoch time).
The call will assign a long integer to a user-designated buffer

###### Parameters

- **lastModifiedTime:** (long) The last modified time. The result of function
  call is returned here


In [16]:
Result<Long> lastModifiedTime = new Result<Long>();

g2engine.getRepositoryLastModifiedTime(lastModifiedTime);

System.out.print(lastModifiedTime.getValue());

1667492085863

## Insert

### Insert parameters

The following variables are used as parameters to the Senzing API.

In [17]:
String dataSourceCode1 = "TEST";
String recordID1 = "1";
String dataSourceCode2 = "TEST";
String recordID2 = "2";
String dataSourceCode3 = "TEST";
String recordID3 = "3";
String dataSourceCode4 = "TEST";
String recordID4 = "4";
String dataSourceCode5 = "TEST";
String recordID5 = "5";
String dataSourceCode6 = "TEST";
String recordID6 = "6";
String dataSourceCode7 = "TEST";
String recordID7 = "7";
String dataSourceCode8 = "TEST";
String recordID8 = "8";

String loadID = null;
long flags=0;

### addRecord

Once the Senzing engine is initialized, use `addRecord()` to load a record
into the Senzing repository
-- `addRecord()` can be called as many times as desired and from multiple
threads at the same time.
The `addRecord()` function returns "0" upon success, and accepts four
parameters as input:

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record
  is associated with.
  This value is configurable to the system
- **recordID:** (str) The record ID, used to identify distinct records
- **jsonData:** (str) A JSON document with the attribute data for the record
- **loadID:** (str) The observation load ID for the record;
  value can be null and will default to data_source

In [18]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"JANE\", \"NAME_LAST\": \"SMITH\", \"ADDR_TYPE\": \"HOME\", \"ADDR_LINE1\": \"653 STATE ROUTE 7\", \"ADDR_CITY\": \"FRESNO\", \"ADDR_STATE\": \"CA\", \"ADDR_POSTAL_CODE\": \"55073-1234\", \"SSN_NUMBER\": \"111-11-1111\"}";
StringBuffer response = new StringBuffer();
String dataSourceCode = dataSourceCode1;
String recordID = recordID1;

int return_code= g2engine.addRecord(
                    dataSourceCode, 
                    recordID, 
                    jsonData, 
                    loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

### addRecordWithReturnedRecordID

Alternatively `addRecordWithReturnedRecordID()` can be used to return the record ID

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record
  is associated with.
  This value is configurable to the system
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.- **record_id:** (str) The record ID, used to identify distinct records
- **jsonData:** (str) A JSON document with the attribute data for the record
- **loadID:** (str) the observation load ID for the record;
  value can be null.

In [19]:
StringBuffer responseBuffer = new StringBuffer();

int return_code = g2engine.addRecordWithReturnedRecordID(
                            dataSourceCode, 
                            responseBuffer, 
                            jsonData, 
                            loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print("New Record ID: " + responseBuffer.toString());

New Record ID: 06C29624478FF1C3285E294E5657796E3A67244A

### addRecordWithInfo

Use if you would like to know what resolved entities were modified when
adding the new record.
It behaves identically to addRecord(),
but returns a json document containing the IDs of the affected entities.

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **recordID:** (str) The record ID, used to identify distinct records
- **jsonData:** (str) A JSON document with the attribute data for the record
- **loadID:** (str) the observation load ID for the record;
  value can be null.
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [21]:
StringBuffer response = new StringBuffer();
int return_code = g2engine.addRecordWithInfo(
                    dataSourceCode, 
                    recordID, 
                    jsonData, 
                    loadID,
                    flags,
                    response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="980fc791-6f4a-47d8-b67b-bd4a0623912e" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('980fc791-6f4a-47d8-b67b-bd4a0623912e').appendChild(renderjson({"DATA_SOURCE":"TEST","RECORD_ID":"1","AFFECTED_ENTITIES":[],"INTERESTING_ENTITIES":{"ENTITIES":[]}}))});

### addRecordWithInfoWithReturnedRecordID


###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **jsonData:** (str) A JSON document with the attribute data for the record
- **loadID:** (str) the observation load ID for the record;
  value can be null.
- **recordResponse:** (StringBuffer) The record ID of the added record.
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [22]:
StringBuffer response = new StringBuffer();
StringBuffer recordResponse = new StringBuffer();

int return_code = g2engine.addRecordWithInfoWithReturnedRecordID(
                    dataSourceCode, 
                    jsonData,
                    loadID,
                    flags,
                    recordResponse,
                    response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(recordResponse);
    RenderJSON(response)

06C29624478FF1C3285E294E5657796E3A67244A

<div id="04d2eb57-8bc5-490f-9083-da19e98c6cf3" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('04d2eb57-8bc5-490f-9083-da19e98c6cf3').appendChild(renderjson({"DATA_SOURCE":"TEST","RECORD_ID":"06C29624478FF1C3285E294E5657796E3A67244A","AFFECTED_ENTITIES":[],"INTERESTING_ENTITIES":{"ENTITIES":[]}}))});

## Replace

### replaceRecord

Use the `replaceRecord()` function to update or replace a record in the data
repository.
If record doesn't exist, a new record is added to the data repository.
Like the above functions, `replaceRecord()` returns "0" upon success,
and it can be called as many times as desired and from multiple threads at
the same time.

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system
- **recordID:** (str) The record ID, used to identify distinct records
- **jsonData:** (str) A JSON document with the attribute data for the record
- **loadID:** (str) the observation load ID for the record;
  value can be null.

In [23]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"JANE\", \"NAME_LAST\": \"ADAMS\", \"ADDR_TYPE\": \"HOME\", \"ADDR_LINE1\": \"653 STATE ROUTE 7\", \"ADDR_CITY\": \"FRESNO\", \"ADDR_STATE\": \"CA\", \"ADDR_POSTAL_CODE\": \"55073-1234\"}";

int return_code= g2engine.replaceRecord(
                    dataSourceCode, 
                    recordID, 
                    jsonData, 
                    loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code)

0

### replaceRecordWithInfo

`replaceRecordWithInfo()` is available if you would like to know what
resolved entities were modified when replacing a record.
It behaves identically to `replaceRecord()`,
but also returns a json document containing the IDs of the affected entities.

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **recordID:** (str) The record ID, used to identify distinct records
- **jsonData:** (str) A JSON document with the attribute data for the record
- **loadID:** (str) the observation load ID for the record;
  value can be null.
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [24]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"JANE\", \"NAME_LAST\": \"ADAMS\", \"ADDR_TYPE\": \"HOME\", \"ADDR_LINE1\": \"653 STATE ROUTE 7\", \"ADDR_CITY\": \"FRESNO\", \"ADDR_STATE\": \"CA\", \"ADDR_POSTAL_CODE\": \"55073-1234\"}";

StringBuffer response = new StringBuffer();

int return_code= g2engine.replaceRecordWithInfo(
                            dataSourceCode, 
                            recordID, 
                            jsonData, 
                            loadID, 
                            flags,
                            response);
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response)

<div id="40dbec8f-548a-4353-8689-a260a6a9a330" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('40dbec8f-548a-4353-8689-a260a6a9a330').appendChild(renderjson({"DATA_SOURCE":"TEST","RECORD_ID":"1","AFFECTED_ENTITIES":[],"INTERESTING_ENTITIES":{"ENTITIES":[]}}))});

## Re-evaluate

### reevaluateRecord

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **recordID:** (str) The record ID, used to identify distinct records

In [25]:
String dataSourceCode = dataSourceCode1;
String recordID = recordID1;

int return_code = g2engine.reevaluateRecord(
                            dataSourceCode,
                            recordID,
                            flags);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code);

0

### reevaluateRecordWithInfo

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **recordID:** (str) The record ID, used to identify distinct records
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [26]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.reevaluateRecordWithInfo(
                            dataSourceCode, 
                            recordID,
                            flags,
                            response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="6dd70146-3f02-431c-9554-3ffd805c1ef3" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('6dd70146-3f02-431c-9554-3ffd805c1ef3').appendChild(renderjson({"DATA_SOURCE":"TEST","RECORD_ID":"1","AFFECTED_ENTITIES":[{"ENTITY_ID":2}],"INTERESTING_ENTITIES":{"ENTITIES":[]}}))});

### reevaluateEntity

- **entityID:** (int) The entity ID for the entity to be analyzed

Find an entity

In [27]:
String dataSourceCode = dataSourceCode1;
String recordID = recordID1;
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(dataSourceCode, recordID, response);
                            
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();

Re-evaluate the entity.

###### Parameters

- **entityID:** (int) The entity ID for the entity to be analyzed

In [28]:
int return_code = g2engine.reevaluateEntity(entityID, flags);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code);

0

### reevaluateEntityWithInfo

###### Parameters

- **entityID:** (int) The entity ID for the entity to be analyzed
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [29]:
StringBuffer response = new StringBuffer();

return_code = g2engine.reevaluateEntityWithInfo(entityID, flags, response);

if(return_code!=0)
    System.out.print(return_code);
else
    RenderJSON(response);

<div id="e2244f79-5b82-452b-bc9f-50e532ddf573" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('e2244f79-5b82-452b-bc9f-50e532ddf573').appendChild(renderjson({"DATA_SOURCE":"TEST","RECORD_ID":"1","AFFECTED_ENTITIES":[{"ENTITY_ID":2}],"INTERESTING_ENTITIES":{"ENTITIES":[]}}))});

## Redo Processing

Redo records are automatically created by Senzing when certain conditions
occur where it believes more processing may be needed.
Some examples:

- A value becomes generic and previous decisions may need to be revisited
- Clean up after some record deletes
- Detected related entities were being changed at the same time
- A table inconsistency exists, potentially after a non-graceful shutdown

First we will need to have a total of 6 data sources so let's add 5 more.

Create Record and Entity for redo

In [30]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Lily\", \"NAME_LAST\": \"Owens\", \"SSN_NUMBER\": \"111-11-1111\"}";

String dataSourceCode = dataSourceCode4;
String recordID = recordID4;
int return_code= g2engine.addRecord(
                    dataSourceCode, 
                    recordID, 
                    jsonData, 
                    loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

In [31]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(dataSourceCode, recordID, response);
                            
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID6 = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();
System.out.println("Entity ID: " + entityID6);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

Entity ID: 3


<div id="0b01d034-cf2e-4d7a-a5ab-1d7155b4ee30" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('0b01d034-cf2e-4d7a-a5ab-1d7155b4ee30').appendChild(renderjson({"RESOLVED_ENTITY":{"ENTITY_ID":3,"ENTITY_NAME":"Lily Owens","FEATURES":{"NAME":[{"FEAT_DESC":"Lily Owens","LIB_FEAT_ID":15,"USAGE_TYPE":"PRIMARY","FEAT_DESC_VALUES":[{"FEAT_DESC":"Lily Owens","LIB_FEAT_ID":15}]}],"SSN":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3,"FEAT_DESC_VALUES":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3}]}]},"RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":1,"FIRST_SEEN_DT":"2022-11-05 16:17:54.941","LAST_SEEN_DT":"2022-11-05 16:17:54.941"}],"LAST_SEEN_DT":"2022-11-05 16:17:54.941","RECORDS":[{"DATA_SOURCE":"TEST","RECORD_ID":"4","ENTITY_TYPE":"GENERIC","INTERNAL_ID":3,"ENTITY_KEY":"C1F081BB9C9EDA71A73873FAF39774DF802A3B9F","ENTITY_DESC":"Lily Owens","MATCH_KEY":"","MATCH_LEVEL":0,"MATCH_LEVEL_CODE":"","ERRULE_CODE":"","LAST_SEEN_DT":"2022-11-05 16:17:54.941"}]},"RELATED_ENTITIES":[]}))});

Create another Record and Entity

In [32]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"August\", \"NAME_LAST\": \"Bauler\", \"NAME_MIDDLE\": \"E\", \"SSN_NUMBER\": \"111-11-1111\"}";

String dataSourceCode = dataSourceCode5;
String recordID = recordID5;
int return_code= g2engine.addRecord(
                    dataSourceCode, 
                    recordID, 
                    jsonData, 
                    loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

In [33]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(dataSourceCode, recordID, response);
                            
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID7 = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();
System.out.println(entityID7);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

4


<div id="f65df011-2f44-4544-a976-0c344b19978b" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('f65df011-2f44-4544-a976-0c344b19978b').appendChild(renderjson({"RESOLVED_ENTITY":{"ENTITY_ID":4,"ENTITY_NAME":"August E Bauler","FEATURES":{"NAME":[{"FEAT_DESC":"August E Bauler","LIB_FEAT_ID":18,"USAGE_TYPE":"PRIMARY","FEAT_DESC_VALUES":[{"FEAT_DESC":"August E Bauler","LIB_FEAT_ID":18}]}],"SSN":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3,"FEAT_DESC_VALUES":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3}]}]},"RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":1,"FIRST_SEEN_DT":"2022-11-05 16:17:57.036","LAST_SEEN_DT":"2022-11-05 16:17:57.036"}],"LAST_SEEN_DT":"2022-11-05 16:17:57.036","RECORDS":[{"DATA_SOURCE":"TEST","RECORD_ID":"5","ENTITY_TYPE":"GENERIC","INTERNAL_ID":4,"ENTITY_KEY":"C9CD56EAFDE58BFE828693F3AE3A0B6BE234599E","ENTITY_DESC":"August E Bauler","MATCH_KEY":"","MATCH_LEVEL":0,"MATCH_LEVEL_CODE":"","ERRULE_CODE":"","LAST_SEEN_DT":"2022-11-05 16:17:57.036"}]},"RELATE

Create another Record and Entity

In [34]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Brian\", \"NAME_LAST\": \"Barcy\", \"NAME_MIDDLE\": \"H\", \"SSN_NUMBER\": \"111-11-1111\"}";

String dataSourceCode = dataSourceCode6;
String recordID = recordID6;
int return_code= g2engine.addRecord(
                    dataSourceCode, 
                    recordID, 
                    jsonData, 
                    loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

In [35]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(dataSourceCode, recordID, response);
                            
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID8 = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();
System.out.println(entityID8);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

5


<div id="c6171af1-e3de-4b1a-8500-50a8ecc95752" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('c6171af1-e3de-4b1a-8500-50a8ecc95752').appendChild(renderjson({"RESOLVED_ENTITY":{"ENTITY_ID":5,"ENTITY_NAME":"Brian H Barcy","FEATURES":{"NAME":[{"FEAT_DESC":"Brian H Barcy","LIB_FEAT_ID":23,"USAGE_TYPE":"PRIMARY","FEAT_DESC_VALUES":[{"FEAT_DESC":"Brian H Barcy","LIB_FEAT_ID":23}]}],"SSN":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3,"FEAT_DESC_VALUES":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3}]}]},"RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":1,"FIRST_SEEN_DT":"2022-11-05 16:17:58.506","LAST_SEEN_DT":"2022-11-05 16:17:58.506"}],"LAST_SEEN_DT":"2022-11-05 16:17:58.506","RECORDS":[{"DATA_SOURCE":"TEST","RECORD_ID":"6","ENTITY_TYPE":"GENERIC","INTERNAL_ID":5,"ENTITY_KEY":"483377E67EF630768293483D48E2073804096612","ENTITY_DESC":"Brian H Barcy","MATCH_KEY":"","MATCH_LEVEL":0,"MATCH_LEVEL_CODE":"","ERRULE_CODE":"","LAST_SEEN_DT":"2022-11-05 16:17:58.506"}]},"RELATED_ENTITI

Create another Record and Entity

In [36]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Jack\", \"NAME_LAST\": \"Miller\", \"NAME_MIDDLE\": \"H\", \"SSN_NUMBER\": \"111-11-1111\"}";

String dataSourceCode = dataSourceCode7;
String recordID = recordID7;
int return_code= g2engine.addRecord(
                    dataSourceCode, 
                    recordID, 
                    jsonData, 
                    loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

In [37]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(dataSourceCode, recordID, response);
                            
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID9 = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();
System.out.println(entityID9);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

6


<div id="a8b551a5-eaf2-4c23-8e45-f62535047184" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('a8b551a5-eaf2-4c23-8e45-f62535047184').appendChild(renderjson({"RESOLVED_ENTITY":{"ENTITY_ID":6,"ENTITY_NAME":"Jack H Miller","FEATURES":{"NAME":[{"FEAT_DESC":"Jack H Miller","LIB_FEAT_ID":28,"USAGE_TYPE":"PRIMARY","FEAT_DESC_VALUES":[{"FEAT_DESC":"Jack H Miller","LIB_FEAT_ID":28}]}],"SSN":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3,"FEAT_DESC_VALUES":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3}]}]},"RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":1,"FIRST_SEEN_DT":"2022-11-05 16:18:00.063","LAST_SEEN_DT":"2022-11-05 16:18:00.063"}],"LAST_SEEN_DT":"2022-11-05 16:18:00.063","RECORDS":[{"DATA_SOURCE":"TEST","RECORD_ID":"7","ENTITY_TYPE":"GENERIC","INTERNAL_ID":6,"ENTITY_KEY":"F5F3629B1E3A8AFCA29A54E4F6E636CBF3DEE4BC","ENTITY_DESC":"Jack H Miller","MATCH_KEY":"","MATCH_LEVEL":0,"MATCH_LEVEL_CODE":"","ERRULE_CODE":"","LAST_SEEN_DT":"2022-11-05 16:18:00.063"}]},"RELATED_ENTITI

Create another Record and Entity

In [38]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Madelyn\", \"NAME_LAST\": \"Caryn\", \"SSN_NUMBER\": \"111-11-1111\"}";

String dataSourceCode = dataSourceCode8;
String recordID = recordID8;
int return_code= g2engine.addRecord(
                    dataSourceCode, 
                    recordID, 
                    jsonData, 
                    loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

In [39]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(dataSourceCode, recordID, response);
                            
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID10 = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();
System.out.println(entityID10);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

7


<div id="91d96c62-da4d-4ddd-a3e6-a2a07228b283" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('91d96c62-da4d-4ddd-a3e6-a2a07228b283').appendChild(renderjson({"RESOLVED_ENTITY":{"ENTITY_ID":7,"ENTITY_NAME":"Madelyn Caryn","FEATURES":{"NAME":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33,"USAGE_TYPE":"PRIMARY","FEAT_DESC_VALUES":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33}]}],"SSN":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3,"FEAT_DESC_VALUES":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3}]}]},"RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":1,"FIRST_SEEN_DT":"2022-11-05 16:18:01.451","LAST_SEEN_DT":"2022-11-05 16:18:01.451"}],"LAST_SEEN_DT":"2022-11-05 16:18:01.451","RECORDS":[{"DATA_SOURCE":"TEST","RECORD_ID":"8","ENTITY_TYPE":"GENERIC","INTERNAL_ID":7,"ENTITY_KEY":"ED809E0DCED3B9F3A411B5FABF62E2D866DBE02F","ENTITY_DESC":"Madelyn Caryn","MATCH_KEY":"","MATCH_LEVEL":0,"MATCH_LEVEL_CODE":"","ERRULE_CODE":"","LAST_SEEN_DT":"2022-11-05 16:18:01.451"}]},"RELATED_ENTITI

### countRedoRecords

Once the Senzing engine is initialized, use `countRedoRecords()`
to return the remaining internally queued maintenance records in the
Senzing repository.
`countRedoRecords()` takes no arguments and returns <0 for errors.

In [40]:
long return_code = g2engine.countRedoRecords();
System.out.print(return_code);

5

### getRedoRecord

Once the Senzing engine is initialized, 
use `getRedoRecord()` to retrieve the next internally queued maintenance
record into the Senzing repository
-- `getRedoRecord()` can be called as many times as desired and from multiple
threads at the same time but all threads are required to be in the same
process.
`getRedoRecord()` should not be called from multiple processes.
Unlike `processRedoRecord()`, `getRedoRecord()` does not actually process the
record.
To process the record, you would use the G2Engine `process()` function.
The `getRedoRecord()` function returns "0" upon success and an empty response
if there is nothing to do.

###### Parameters

- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [41]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.getRedoRecord(response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response)

<div id="4e195dfe-5eea-4142-9178-7eaaff6e25e8" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('4e195dfe-5eea-4142-9178-7eaaff6e25e8').appendChild(renderjson({"REASON":"LIB_FEAT_ID[10] of FTYPE_ID[20] went generic for CANDIDATES ECLASS_ID[1] in LENS_ID[1]","DATA_SOURCE":"TEST","RECORD_ID":"4","ENTITY_TYPE":"GENERIC","DSRC_ACTION":"X"}))});

### processWithInfo

###### Parameters

- **response:** (StringBuffer) the input record to add
- **processResponse:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [42]:
StringBuffer processResponse = new StringBuffer();

int return_code = g2engine.processWithInfo(
                            response.toString(),
                            flags,
                            processResponse);
                            
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response)

<div id="14d3519f-e746-4324-a5e8-577bf4012ab5" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('14d3519f-e746-4324-a5e8-577bf4012ab5').appendChild(renderjson({"REASON":"LIB_FEAT_ID[10] of FTYPE_ID[20] went generic for CANDIDATES ECLASS_ID[1] in LENS_ID[1]","DATA_SOURCE":"TEST","RECORD_ID":"4","ENTITY_TYPE":"GENERIC","DSRC_ACTION":"X"}))});

### process

###### Parameters

- **response:** (bytearray) the input record to process

In [43]:
int return_code = g2engine.process(response.toString());
                            
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response)

<div id="603d24a0-5902-4db3-b604-9238b18aae4b" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('603d24a0-5902-4db3-b604-9238b18aae4b').appendChild(renderjson({"REASON":"LIB_FEAT_ID[10] of FTYPE_ID[20] went generic for CANDIDATES ECLASS_ID[1] in LENS_ID[1]","DATA_SOURCE":"TEST","RECORD_ID":"4","ENTITY_TYPE":"GENERIC","DSRC_ACTION":"X"}))});

### processRedoRecord

This processes the next redo record and returns it
(If `processRedoRecord()` "response" returns 0
and "response_bytearray" is blank then there are no more redo records to
process and if you do `countRedoRecords()` again it will return 0)
Has potential to create more redo records in certian situations.

###### Parameters

- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [44]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.processRedoRecord(response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response)

<div id="d8e3f79b-9998-40bf-b120-84e0e89aeeba" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('d8e3f79b-9998-40bf-b120-84e0e89aeeba').appendChild(renderjson({"REASON":"LIB_FEAT_ID[10] of FTYPE_ID[20] went generic for CANDIDATES ECLASS_ID[1] in LENS_ID[1]","DATA_SOURCE":"TEST","RECORD_ID":"06C29624478FF1C3285E294E5657796E3A67244A","ENTITY_TYPE":"GENERIC","DSRC_ACTION":"X"}))});

### processRedoRecordWithInfo

`processRedoRecordWithInfo()` is available if you would like to know what
resolved entities were modified when processing a redo record.
It behaves identically to `processRedoRecord()`,
but also returns a json document containing the IDs of the affected entities.
It accepts the following parameters:

###### Parameters

- **info:** (StringBuffer) Returns the "withInfo" section 
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [45]:
StringBuffer response = new StringBuffer();
StringBuffer info = new StringBuffer();

int return_code = g2engine.processRedoRecordWithInfo(
    flags,
    info,
    response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);
    RenderJSON(info)

<div id="47e12280-fddc-43c1-b8b8-a0ea15b4892e" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('47e12280-fddc-43c1-b8b8-a0ea15b4892e').appendChild(renderjson({"DATA_SOURCE":"TEST","RECORD_ID":"5","AFFECTED_ENTITIES":[{"ENTITY_ID":4}],"INTERESTING_ENTITIES":{"ENTITIES":[]}}))});

<div id="130ad8ca-27b5-438b-8efd-f4927f2c87bd" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('130ad8ca-27b5-438b-8efd-f4927f2c87bd').appendChild(renderjson({"REASON":"LIB_FEAT_ID[10] of FTYPE_ID[20] went generic for CANDIDATES ECLASS_ID[1] in LENS_ID[1]","DATA_SOURCE":"TEST","RECORD_ID":"5","ENTITY_TYPE":"GENERIC","DSRC_ACTION":"X"}))});

## Delete

### deleteRecord

Use `deleteRecord()` to remove a record from the data repository
(returns "0" upon success);
`deleteRecord()` can be called as many times as desired and from multiple
threads at the same time.

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **recordID:** (str) The record ID, used to identify distinct records
- **loadID:** (str) the observation load ID for the record;
  value can be null.

In [46]:
String dataSourceCode = dataSourceCode1;
String recordID = recordID1;

int return_code= g2engine.deleteRecord(dataSourceCode, recordID, loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code);

0

### deleteRecordWithInfo

`deleteRecordWithInfo()` behaves the same as `deleteRecord()`
but also returns a json document containing the IDs of the affected entities.

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with. This value is configurable to the system.
- **recordID:** (str) The record ID, used to identify distinct records.
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.
- **loadID:** (str) the observation load ID for the record;
  value can be null.


In [47]:
String dataSourceCode = dataSourceCode2;
String recordID = recordID2;
StringBuffer response = new StringBuffer();

int return_code = g2engine.deleteRecordWithInfo(
                            dataSourceCode, 
                            recordID, 
                            loadID, 
                            flags,
                            response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="f3644377-b01f-40f0-aed8-2cfb662c2e45" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('f3644377-b01f-40f0-aed8-2cfb662c2e45').appendChild(renderjson({"DATA_SOURCE":"TEST","RECORD_ID":"2","AFFECTED_ENTITIES":[],"INTERESTING_ENTITIES":{"ENTITIES":[]}}))});

Attempt to get the record again. It should error and give an output similar to "Unknown record"

In [48]:
StringBuffer response = new StringBuffer();
int return_code = g2engine.getRecord(dataSourceCode, recordID, response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

0033E|Unknown record: dsrc[TEST], record[2]

Add record back for later API calls

In [49]:
String dataSourceCode = dataSourceCode1;
String recordID = recordID1;

int return_code = g2engine.addRecord(dataSourceCode, recordID, jsonData, loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

String dataSourceCode = dataSourceCode2;
String recordID = recordID2;

int return_code = g2engine.addRecord(dataSourceCode, recordID, jsonData, loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

## Search

### Record search

#### getRecord

Use `getRecord()` to retrieve a single record from the data repository;
the record is assigned in JSON form to a user-designated buffer.
Once the Senzing engine is initialized,
`getRecord()` can be called as many times as desired and from multiple
threads at the same time.

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **recordID:** (str) The record ID, used to identify the record for retrieval
- **g2EngineFlags:** (long) control flags (applicable flags listed here [getRecord flags](senzing-G2Engine-flags.ipynb#getRecord))
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [50]:
String dataSourceCode = "TEST";
String recordID = "1";
StringBuffer response = new StringBuffer();
int return_code= g2engine.getRecord(
                    dataSourceCode, 
                    recordID, 
                    response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response)

<div id="e6158bc5-7f48-4167-adc3-3ff53a690690" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('e6158bc5-7f48-4167-adc3-3ff53a690690').appendChild(renderjson({"DATA_SOURCE":"TEST","RECORD_ID":"1","JSON_DATA":{"NAME_TYPE":"PRIMARY","NAME_FIRST":"Madelyn","NAME_LAST":"Caryn","SSN_NUMBER":"111-11-1111","DATA_SOURCE":"TEST","ENTITY_TYPE":"GENERIC","DSRC_ACTION":"A","RECORD_ID":"1"}}))});

### Entity search

#### getEntityByRecordID

Entity searching is a key component for interactive use of Entity Resolution
intelligence.
The core Senzing engine provides real-time search capabilities that are
easily accessed via the Senzing API.
Senzing offers methods for entity searching, all of which can be called
as many times as desired and from multiple threads at the same time
(and all of which return "0" upon success).

Use `getEntityByRecordID()` to retrieve entity data based on the ID of a
resolved identity.

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record is associated
  with.  This value is configurable to the system.
- **recordID:** (str) The record ID, used to identify the record for retrieval
- **g2EngineFlags:** (long) control flags (applicable flags listed here [getEntityByRecordID flags](senzing-G2Engine-flags.ipynb#getEntityByRecordID))
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

In [51]:
String dataSourceCode = "TEST";
String recordID = "1";

StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(
                            dataSourceCode, 
                            recordID, 
                            response);

JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID1 = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();

if(return_code!=0)
    RenderJSON(g2engine.getLastException());
else
    RenderJSON(response);

<div id="17e553aa-450f-43ac-9ae2-746bb372705a" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('17e553aa-450f-43ac-9ae2-746bb372705a').appendChild(renderjson({"RESOLVED_ENTITY":{"ENTITY_ID":7,"ENTITY_NAME":"Madelyn Caryn","FEATURES":{"NAME":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33,"USAGE_TYPE":"PRIMARY","FEAT_DESC_VALUES":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33}]}],"SSN":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3,"FEAT_DESC_VALUES":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3}]}]},"RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":3,"FIRST_SEEN_DT":"2022-11-05 16:18:01.451","LAST_SEEN_DT":"2022-11-05 16:18:13.899"}],"LAST_SEEN_DT":"2022-11-05 16:18:13.899","RECORDS":[{"DATA_SOURCE":"TEST","RECORD_ID":"1","ENTITY_TYPE":"GENERIC","INTERNAL_ID":7,"ENTITY_KEY":"ED809E0DCED3B9F3A411B5FABF62E2D866DBE02F","ENTITY_DESC":"Madelyn Caryn","MATCH_KEY":"","MATCH_LEVEL":0,"MATCH_LEVEL_CODE":"","ERRULE_CODE":"","LAST_SEEN_DT":"2022-11-05 16:18:13.745"},{"DATA_SOURCE":"T

#### getEntityByEntityID

Use `getEntityByEntityID()` to retrieve entity data based on the ID of a
resolved identity.

###### Parameters

- **entityID:** (long) The numeric ID of a resolved entity
- **g2EngineFlags:** (long) control flags (applicable flags listed here [getEntityByEntityID flags](senzing-G2Engine-flags.ipynb#getEntityByEntityID))
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later

In [52]:
long entityID = entityID1;
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByEntityID(entityID, response);

if(return_code!=0)
    RenderJSON(g2engine.getLastException());
else
    RenderJSON(response);

<div id="0dbefa66-59cf-4c9f-95f5-3cd151d6096e" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('0dbefa66-59cf-4c9f-95f5-3cd151d6096e').appendChild(renderjson({"RESOLVED_ENTITY":{"ENTITY_ID":7,"ENTITY_NAME":"Madelyn Caryn","FEATURES":{"NAME":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33,"USAGE_TYPE":"PRIMARY","FEAT_DESC_VALUES":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33}]}],"SSN":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3,"FEAT_DESC_VALUES":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3}]}]},"RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":3,"FIRST_SEEN_DT":"2022-11-05 16:18:01.451","LAST_SEEN_DT":"2022-11-05 16:18:13.899"}],"LAST_SEEN_DT":"2022-11-05 16:18:13.899","RECORDS":[{"DATA_SOURCE":"TEST","RECORD_ID":"1","ENTITY_TYPE":"GENERIC","INTERNAL_ID":7,"ENTITY_KEY":"ED809E0DCED3B9F3A411B5FABF62E2D866DBE02F","ENTITY_DESC":"Madelyn Caryn","MATCH_KEY":"","MATCH_LEVEL":0,"MATCH_LEVEL_CODE":"","ERRULE_CODE":"","LAST_SEEN_DT":"2022-11-05 16:18:13.745"},{"DATA_SOURCE":"T

#### searchByAttributes

This function is similar but preferable to the searchByAttributes() function.

Use `searchByAttributes()` to retrieve entity data based on
a user-specified set of entity attributes.

###### Parameters

- **jsonData:** (str) A JSON document with the attribute data to search for.
- **g2EngineFlags:** (long) control flags (applicable flags listed here [searchByAttributes flags](senzing-G2Engine-flags.ipynb#searchByAttributes))
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later

In [53]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.searchByAttributes(jsonData, response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="6629d613-2cef-4efd-830f-38820da8be3a" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('6629d613-2cef-4efd-830f-38820da8be3a').appendChild(renderjson({"RESOLVED_ENTITIES":[{"MATCH_INFO":{"MATCH_LEVEL":4,"MATCH_LEVEL_CODE":"NAME_ONLY","MATCH_KEY":"+NAME+SSN","ERRULE_CODE":"CNAME","FEATURE_SCORES":{"NAME":[{"INBOUND_FEAT":"Madelyn Caryn","CANDIDATE_FEAT":"Madelyn Caryn","GNR_FN":100,"GNR_SN":100,"GNR_GN":100,"GENERATION_MATCH":-1,"GNR_ON":-1}],"SSN":[{"INBOUND_FEAT":"111-11-1111","CANDIDATE_FEAT":"111-11-1111","FULL_SCORE":100}]}},"ENTITY":{"RESOLVED_ENTITY":{"ENTITY_ID":7,"ENTITY_NAME":"Madelyn Caryn","FEATURES":{"NAME":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33,"USAGE_TYPE":"PRIMARY","FEAT_DESC_VALUES":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33}]}],"SSN":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3,"FEAT_DESC_VALUES":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3}]}]},"RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":3,"FIRST_SEEN_DT":"2022-11-05 16:18:01

#### getVirtualEntityByRecordID

###### Parameters

- **records:** (str) A JSON document with the datasource codes and recordID's of the records to check against
- **g2EngineFlags:** (long) control flags (applicable flags listed here [getVirtualEntityByRecordID flags](senzing-G2Engine-flags.ipynb#getVirtualEntityByRecordID))
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later

In [54]:
StringBuffer response = new StringBuffer();
String records =  "{\"RECORDS\": [{\"DATA_SOURCE\": \""+dataSourceCode1+"\", \"RECORD_ID\": \""+recordID1+"\"}, {\"DATA_SOURCE\": \""+dataSourceCode2+"\", \"RECORD_ID\": \""+recordID2+"\"}]}";

int return_code = g2engine.getVirtualEntityByRecordID(
                            records,
                            response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="5f731e04-b731-411b-973e-bdf366d02df6" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('5f731e04-b731-411b-973e-bdf366d02df6').appendChild(renderjson({"RESOLVED_ENTITY":{"ENTITY_ID":7,"ENTITY_NAME":"Madelyn Caryn","FEATURES":{"ID_KEY":[{"FEAT_DESC":"SSN=111-11-1111","LIB_FEAT_ID":10,"FEAT_DESC_VALUES":[{"FEAT_DESC":"SSN=111-11-1111","LIB_FEAT_ID":10,"USED_FOR_CAND":"Y","USED_FOR_SCORING":"N","ENTITY_COUNT":6,"CANDIDATE_CAP_REACHED":"Y","SCORING_CAP_REACHED":"N","SUPPRESSED":"N"}]}],"NAME":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33,"USAGE_TYPE":"PRIMARY","FEAT_DESC_VALUES":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33,"USED_FOR_CAND":"N","USED_FOR_SCORING":"Y","ENTITY_COUNT":1,"CANDIDATE_CAP_REACHED":"N","SCORING_CAP_REACHED":"N","SUPPRESSED":"N"}]}],"NAME_KEY":[{"FEAT_DESC":"KRN|MTLN","LIB_FEAT_ID":34,"FEAT_DESC_VALUES":[{"FEAT_DESC":"KRN|MTLN","LIB_FEAT_ID":34,"USED_FOR_CAND":"Y","USED_FOR_SCORING":"N","ENTITY_COUNT":1,"CANDIDATE_CAP_REACHED":"N","SCORING_CA

### Finding Paths

First you will need to create some records so that you have some that you can
compare.
Can you see what is the same between this record and the previous one?

#### Create records for paths

In [55]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Max\", \"NAME_LAST\": \"Miller\", \"NAME_MIDDLE\": \"W\",\"SSN_NUMBER\": \"111-11-1111\"}";

String dataSourceCode = dataSourceCode2;
String recordID = recordID2;
int return_code = g2engine.replaceRecord(dataSourceCode, recordID, jsonData, loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

Replace values for Record #3

In [56]:
String jsonData =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Mildred\", \"NAME_LAST\": \"Miller\", \"SSN_NUMBER\": \"111-11-1111\"}";

String dataSourceCode = dataSourceCode3;
String recordID = recordID3;
int return_code = g2engine.replaceRecord(dataSourceCode, recordID, jsonData, loadID);

if(return_code!=0)
    System.out.print(g2engine.getLastException());

Locate "entity identifier" for Record #1

In [57]:
String dataSourceCode = dataSourceCode1;
String recordID = recordID1;
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(
                            dataSourceCode,
                            recordID,
                            response);
                            
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID2 = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="f0ce0167-0396-4fbc-83e0-bc336e58afeb" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('f0ce0167-0396-4fbc-83e0-bc336e58afeb').appendChild(renderjson({"RESOLVED_ENTITY":{"ENTITY_ID":7,"ENTITY_NAME":"Madelyn Caryn","FEATURES":{"NAME":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33,"USAGE_TYPE":"PRIMARY","FEAT_DESC_VALUES":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33}]}],"SSN":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3,"FEAT_DESC_VALUES":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3}]}]},"RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":2,"FIRST_SEEN_DT":"2022-11-05 16:18:01.451","LAST_SEEN_DT":"2022-11-05 16:18:13.745"}],"LAST_SEEN_DT":"2022-11-05 16:18:13.745","RECORDS":[{"DATA_SOURCE":"TEST","RECORD_ID":"1","ENTITY_TYPE":"GENERIC","INTERNAL_ID":7,"ENTITY_KEY":"ED809E0DCED3B9F3A411B5FABF62E2D866DBE02F","ENTITY_DESC":"Madelyn Caryn","MATCH_KEY":"","MATCH_LEVEL":0,"MATCH_LEVEL_CODE":"","ERRULE_CODE":"","LAST_SEEN_DT":"2022-11-05 16:18:13.745"},{"DATA_SOURCE":"T

Locate "entity identifier" for Record #3

In [58]:
String dataSourceCode = dataSourceCode3;
String recordID = recordID3;
StringBuffer response = new StringBuffer();

int return_code = g2engine.getEntityByRecordID(
                            dataSourceCode,
                            recordID,
                            response);
                            
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID3 = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="e171094d-ca1b-4e3a-aadb-1356765a314e" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('e171094d-ca1b-4e3a-aadb-1356765a314e').appendChild(renderjson({"RESOLVED_ENTITY":{"ENTITY_ID":9,"ENTITY_NAME":"Mildred Miller","FEATURES":{"NAME":[{"FEAT_DESC":"Mildred Miller","LIB_FEAT_ID":41,"USAGE_TYPE":"PRIMARY","FEAT_DESC_VALUES":[{"FEAT_DESC":"Mildred Miller","LIB_FEAT_ID":41}]}],"SSN":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3,"FEAT_DESC_VALUES":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3}]}]},"RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":1,"FIRST_SEEN_DT":"2022-11-05 16:18:22.340","LAST_SEEN_DT":"2022-11-05 16:18:22.340"}],"LAST_SEEN_DT":"2022-11-05 16:18:22.340","RECORDS":[{"DATA_SOURCE":"TEST","RECORD_ID":"3","ENTITY_TYPE":"GENERIC","INTERNAL_ID":9,"ENTITY_KEY":"B4C5F2D4B888C71C115DE44C8E6389FC57B6B2C5","ENTITY_DESC":"Mildred Miller","MATCH_KEY":"","MATCH_LEVEL":0,"MATCH_LEVEL_CODE":"","ERRULE_CODE":"","LAST_SEEN_DT":"2022-11-05 16:18:22.340"}]},"RELATED_EN

#### findPathByEntityID

The `findPathByEntityID()` function can be used to find single relationship paths between two entities. Paths are found using known relationships with other entities.

###### Parameters

- **entityIDStart:** (int) The entity ID for the starting entity of the search path
- **entityIDEnd:** (int) The entity ID for the ending entity of the search path
- **maxDegree:** (int) The number of relationship degrees to search
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later
- **g2EngineFlags:** (long) control flags (applicable flags listed here [findPathByEntityID flags](senzing-G2Engine-flags.ipynb#findPathByEntityID))

The functions return a JSON document that identifies the path between the
entities,
and the information on the entities in question.
The document contains a section called "ENTITY_PATHS" which gives
the path from one entity to the other.

In [59]:
long entityIDStart = entityID2;
long entityIDEnd = entityID3;

int maxDegree = 3;

StringBuffer response = new StringBuffer();

int ret_code = g2engine.findPathByEntityID(
                            entityIDStart,
                            entityIDEnd,
                            maxDegree, 
                            response);

if(ret_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="0e8865fa-690b-4f9d-960f-e800fc64a561" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('0e8865fa-690b-4f9d-960f-e800fc64a561').appendChild(renderjson({"ENTITY_PATHS":[{"START_ENTITY_ID":7,"END_ENTITY_ID":9,"ENTITIES":[]}],"ENTITIES":[{"RESOLVED_ENTITY":{"ENTITY_ID":7,"ENTITY_NAME":"Madelyn Caryn","RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":2,"FIRST_SEEN_DT":"2022-11-05 16:18:01.451","LAST_SEEN_DT":"2022-11-05 16:18:13.745"}],"LAST_SEEN_DT":"2022-11-05 16:18:13.745"},"RELATED_ENTITIES":[]},{"RESOLVED_ENTITY":{"ENTITY_ID":9,"ENTITY_NAME":"Mildred Miller","RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":1,"FIRST_SEEN_DT":"2022-11-05 16:18:22.340","LAST_SEEN_DT":"2022-11-05 16:18:22.340"}],"LAST_SEEN_DT":"2022-11-05 16:18:22.340"},"RELATED_ENTITIES":[]}]}))});

#### findPathByRecordID

The `findPathByRecordID()` function
can be used to find single relationship paths between two entities.
Paths are found using known relationships with other entities.

###### Parameters

- **dataSourceCodeStart:** (str) The data source for the starting entity of the
  search path
- **dataSourceCodeEnd:** (str) The data source for the ending entity of the search
  path
- **recordIDStart:** (int) The record ID for the starting entity of the search path
- **recordIDEnd:** (int) The record ID for the ending entity of the search path
- **maxDegree:** (int) The number of relationship degrees to search
- **g2EngineFlags:** (long) control flags (applicable flags listed here [findPathByRecordID flags](senzing-G2Engine-flags.ipynb#findPathByRecordID))
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

The functions return a JSON document that identifies the path between the
entities,
and the information on the entities in question.
The document contains a section called "ENTITY_PATHS" which gives
the path from one entity to the other.

In [60]:
String dataSourceCodeStart = dataSourceCode2;
String dataSourceCodeEnd = dataSourceCode3;
String recordIDStart = recordID2;
String recordIDEnd = recordID3;
int maxDegree = 3;

ret_code = g2engine.findPathByRecordID(
                        dataSourceCodeStart,
                        recordIDStart,
                        dataSourceCodeEnd,
                        recordIDEnd,
                        maxDegree,
                        response);

if(ret_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="11788347-3d87-4bee-acb9-9682fb5ded05" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('11788347-3d87-4bee-acb9-9682fb5ded05').appendChild(renderjson({"ENTITY_PATHS":[{"START_ENTITY_ID":8,"END_ENTITY_ID":9,"ENTITIES":[]}],"ENTITIES":[{"RESOLVED_ENTITY":{"ENTITY_ID":8,"ENTITY_NAME":"Max W Miller","RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":1,"FIRST_SEEN_DT":"2022-11-05 16:18:13.899","LAST_SEEN_DT":"2022-11-05 16:18:21.309"}],"LAST_SEEN_DT":"2022-11-05 16:18:21.309"},"RELATED_ENTITIES":[]},{"RESOLVED_ENTITY":{"ENTITY_ID":9,"ENTITY_NAME":"Mildred Miller","RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":1,"FIRST_SEEN_DT":"2022-11-05 16:18:22.340","LAST_SEEN_DT":"2022-11-05 16:18:22.340"}],"LAST_SEEN_DT":"2022-11-05 16:18:22.340"},"RELATED_ENTITIES":[]}]}))});

#### findPathExcludingByEntityID

The `findPathExcludingByEntityID()` function can be used to find single relationship paths between two
entities.
Paths are found using known relationships with other entities.
In addition, it will find paths that exclude certain entities from being on
the path.


When excluding entities, the user may choose to either (a) strictly exclude
the entities,
or (b) prefer to exclude the entities, but still include them if no other
path is found.
By default, entities will be strictly excluded.
A "preferred exclude" may be done by specifying the
`G2_FIND_PATH_PREFER_EXCLUDE` control flag.

###### Parameters

- **entityIDStart:** (int) The entity ID for the starting entity of the search path
- **entityIDEnd:** (int) The entity ID for the ending entity of the search path
- **maxDegree:** (int) The number of relationship degrees to search
- **excludedEntities:** (str) Entities that should be avoided on the path
  (JSON document)
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later
- **g2EngineFlags:** (long) control flags (applicable flags listed here [findPathExcludingByEntityID flags](senzing-G2Engine-flags.ipynb#findPathExcludingByEntityID))

In [61]:
int maxDegree = 4;
String excludedEntities = new String("{\"ENTITIES\":[{\"ENTITY_ID\":\"1\"}]}");

StringBuffer response = new StringBuffer();

long entityIDStart = entityID2;
long entityIDEnd = entityID3;

int return_code = g2engine.findPathExcludingByEntityID(
                            entityIDStart,
                            entityIDEnd,
                            maxDegree,
                            excludedEntities,
                            response);

if(ret_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="400b837b-dccc-447d-b1e1-d654e7d84b07" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('400b837b-dccc-447d-b1e1-d654e7d84b07').appendChild(renderjson({"ENTITY_PATHS":[{"START_ENTITY_ID":7,"END_ENTITY_ID":9,"ENTITIES":[]}],"ENTITIES":[{"RESOLVED_ENTITY":{"ENTITY_ID":7,"ENTITY_NAME":"Madelyn Caryn","RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":2,"FIRST_SEEN_DT":"2022-11-05 16:18:01.451","LAST_SEEN_DT":"2022-11-05 16:18:13.745"}],"LAST_SEEN_DT":"2022-11-05 16:18:13.745"},"RELATED_ENTITIES":[]},{"RESOLVED_ENTITY":{"ENTITY_ID":9,"ENTITY_NAME":"Mildred Miller","RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":1,"FIRST_SEEN_DT":"2022-11-05 16:18:22.340","LAST_SEEN_DT":"2022-11-05 16:18:22.340"}],"LAST_SEEN_DT":"2022-11-05 16:18:22.340"},"RELATED_ENTITIES":[]}]}))});

#### findPathExcludingByRecordID

The `findPathExcludingByRecordID()`
function can be used to find single relationship paths between two
entities.
Paths are found using known relationships with other entities.
In addition, it will find paths that exclude certain entities from being on
the path.

When excluding entities, the user may choose to either (a) strictly exclude
the entities,
or (b) prefer to exclude the entities, but still include them if no other
path is found.
By default, entities will be strictly excluded.
A "preferred exclude" may be done by specifying the
`G2_FIND_PATH_PREFER_EXCLUDE` control flag.

###### Parameters

- **dataSourceCodeStart:** (str) The data source for the starting entity of the
  search path
- **dataSourceCodeEnd:** (str) The data source for the ending entity of the search
  path
- **recordIDStart:** (str) The record ID for the starting entity of the search path
- **recordIDEnd:** (str) The record ID for the ending entity of the search path
- **maxDegree:** (int) The number of relationship degrees to search
- **excludedEntities:** (str) Entities that should be avoided on the path
  (JSON document)
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.
- **g2EngineFlags:** (long) control flags (applicable flags listed here [findPathExcludingByRecordID flags](senzing-G2Engine-flags.ipynb#findPathExcludingByRecordID))


In [62]:
String dataSourceCodeStart = dataSourceCode2;
String dataSourceCodeEnd = dataSourceCode3;
String recordIDStart = recordID2;
String recordIDEnd = recordID3;
StringBuffer response = new StringBuffer();

String excludedRecords = new String("{\"RECORDS\":[{\"RECORD_ID\":\"1\",\"DATA_SOURCE\":\"TEST\"}]}");

return_code = g2engine.findPathExcludingByRecordID(
                        dataSourceCodeStart,
                        recordIDStart,
                        dataSourceCodeEnd,
                        recordIDEnd,
                        maxDegree,
                        excludedRecords,
                        response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="0dc63ae2-8a65-43e4-8b18-e6ac69eccdc9" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('0dc63ae2-8a65-43e4-8b18-e6ac69eccdc9').appendChild(renderjson({"ENTITY_PATHS":[{"START_ENTITY_ID":8,"END_ENTITY_ID":9,"ENTITIES":[]}],"ENTITIES":[{"RESOLVED_ENTITY":{"ENTITY_ID":8,"ENTITY_NAME":"Max W Miller","RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":1,"FIRST_SEEN_DT":"2022-11-05 16:18:13.899","LAST_SEEN_DT":"2022-11-05 16:18:21.309"}],"LAST_SEEN_DT":"2022-11-05 16:18:21.309"},"RELATED_ENTITIES":[]},{"RESOLVED_ENTITY":{"ENTITY_ID":9,"ENTITY_NAME":"Mildred Miller","RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":1,"FIRST_SEEN_DT":"2022-11-05 16:18:22.340","LAST_SEEN_DT":"2022-11-05 16:18:22.340"}],"LAST_SEEN_DT":"2022-11-05 16:18:22.340"},"RELATED_ENTITIES":[]}]}))});

### Finding Paths with Required Sources

#### findPathIncludingSourceByEntityID

The `findPathIncludingSourceByEntityID()` function
can be used to find single relationship paths between two entities.
In addition, one of the enties along the path must include a specified data
source.

Specific entities may also be excluded, using the same methodology as the
`findPathExcludingByEntityID()` and `findPathExcludingByRecordID()`
functions use.

###### Parameters

- **entityIDStart:** (int) The entity ID for the starting entity of the search path
- **entityIDEnd:** (int) The entity ID for the ending entity of the search path
- **maxDegree:** (int) The number of relationship degrees to search
- **excludedEntities:** (str) Entities that should be avoided on the path
  (JSON document)
- **requiredDsrcs:** (str) Entities that should be avoided on the path
  (JSON document)
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later
- **g2EngineFlags:** (long) control flags (applicable flags listed here [findPathIncludingSourceByEntityID flags](senzing-G2Engine-flags.ipynb#findPathIncludingSourceByEntityID))


In [63]:
int maxDegree = 4;
String excludedEntities = new String("{\"ENTITIES\":[{\"ENTITY_ID\":\"1\"}]}");
String requiredDsrcs = new String("{\"DATA_SOURCES\":[\"TEST\"]}");

StringBuffer response = new StringBuffer();

int ret_code = g2engine.findPathIncludingSourceByEntityID(
                            entityIDStart,
                            entityIDEnd,
                            maxDegree,
                            excludedEntities,
                            requiredDsrcs,
                            response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="19060c56-d7ab-49d7-acf0-6bc88bbefea6" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('19060c56-d7ab-49d7-acf0-6bc88bbefea6').appendChild(renderjson({"ENTITY_PATHS":[{"START_ENTITY_ID":7,"END_ENTITY_ID":9,"ENTITIES":[]}],"ENTITIES":[{"RESOLVED_ENTITY":{"ENTITY_ID":7,"ENTITY_NAME":"Madelyn Caryn","RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":2,"FIRST_SEEN_DT":"2022-11-05 16:18:01.451","LAST_SEEN_DT":"2022-11-05 16:18:13.745"}],"LAST_SEEN_DT":"2022-11-05 16:18:13.745"},"RELATED_ENTITIES":[]},{"RESOLVED_ENTITY":{"ENTITY_ID":9,"ENTITY_NAME":"Mildred Miller","RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":1,"FIRST_SEEN_DT":"2022-11-05 16:18:22.340","LAST_SEEN_DT":"2022-11-05 16:18:22.340"}],"LAST_SEEN_DT":"2022-11-05 16:18:22.340"},"RELATED_ENTITIES":[]}]}))});

#### findPathIncludingSourceByRecordID


The 
`findPathIncludingSourceByRecordID()` function
can be used to find single relationship paths between two entities.
In addition, one of the enties along the path must include a specified data
source.

Specific entities may also be excluded, using the same methodology as the
`findPathExcludingByEntityID()` and `findPathExcludingByRecordID()`
functions use.

###### Parameters

- **dataSourceCodeStart:** (str) The data source for the starting entity of the
  search path
- **dataSourceCodeEnd:** (str) The data source for the ending entity of the search
  path
- **recordIDStart:** (str) The record ID for the starting entity of the search
  path
- **recordIDEnd:** (str) The record ID for the ending entity of the search path
- **maxDegree:** (int) The number of relationship degrees to search
- **excludedEntities:** (str) Entities that should be avoided on the path
  (JSON document)
- **requiredDsrcs:** (str) Entities that should be avoided on the path
  (JSON document)
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later
- **g2EngineFlags:** (long) control flags (applicable flags listed here [findPathIncludingSourceByRecordID flags](senzing-G2Engine-flags.ipynb#findPathIncludingSourceByRecordID))

In [64]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.findPathIncludingSourceByRecordID(
                            dataSourceCodeStart,
                            recordIDStart,
                            dataSourceCodeEnd,
                            recordIDEnd,
                            maxDegree,
                            excludedEntities,
                            requiredDsrcs,
                            response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="04769f5a-2755-467b-aee0-7f144f3737d7" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('04769f5a-2755-467b-aee0-7f144f3737d7').appendChild(renderjson({"ENTITY_PATHS":[{"START_ENTITY_ID":8,"END_ENTITY_ID":9,"ENTITIES":[]}],"ENTITIES":[{"RESOLVED_ENTITY":{"ENTITY_ID":8,"ENTITY_NAME":"Max W Miller","RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":1,"FIRST_SEEN_DT":"2022-11-05 16:18:13.899","LAST_SEEN_DT":"2022-11-05 16:18:21.309"}],"LAST_SEEN_DT":"2022-11-05 16:18:21.309"},"RELATED_ENTITIES":[]},{"RESOLVED_ENTITY":{"ENTITY_ID":9,"ENTITY_NAME":"Mildred Miller","RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":1,"FIRST_SEEN_DT":"2022-11-05 16:18:22.340","LAST_SEEN_DT":"2022-11-05 16:18:22.340"}],"LAST_SEEN_DT":"2022-11-05 16:18:22.340"},"RELATED_ENTITIES":[]}]}))});

### Finding Networks

The `findNetworkByEntityID()` and `findNetworkByRecordID()` functions
can be used to find all entities surrounding a requested set of entities.
This includes the requested entities, paths between them, and relations to
other nearby entities.

Entities can be searched for by either Entity ID or by Record ID,
depending on which function is chosen.

These functions have the following parameters:

- **entityList:** (str) A list of entities, specified by Entity ID
  (JSON document)
- **recordList:** (str) A list of entities, specified by Record ID
  (JSON document)
- **maxDegree:** (int) The maximum number of degrees in paths between search
  entities
- **buildoutDegree:** (int) The number of degrees of relationships to show around
  each search entity
- **maxEntities:** (int) The maximum number of entities to return in the
  discovered network
- **flags:** (int) Control flags for specifying what data about the
  entity to retrieve.
- **response:** (StringBuffer) A memory buffer for returning the response
  document; if an error occurred, an error response is stored here.
  
They also have various arguments used to return response documents

The functions return a JSON document that identifies the path between the
each set of search entities (if the path exists), and the information on the
entities in question (search entities, path entities, and build-out entities.

#### findNetworkByEntityID

The `findNetworkByEntityID()` function
can be used to find all entities surrounding a requested set of entities.
This includes the requested entities, paths between them, and relations to
other nearby entities.

###### Parameters

- **maxDegree:** (int) The maximum number of degrees in paths between search
  entities
- **buildoutDegree:** (int) The number of degrees of relationships to show around
  each search entity
- **maxEntities:** (int) The maximum number of entities to return in the
  discovered network
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later
- **g2EngineFlags:** (long) control flags (applicable flags listed here [findNetworkByEntityID flags](senzing-G2Engine-flags.ipynb#findNetworkByEntityID))

The functions return a JSON document that identifies the path between the
each set of search entities (if the path exists), and the information on the
entities in question (search entities, path entities, and build-out entities.

In [65]:
StringBuffer response = new StringBuffer();
int maxDegree = 2;
int buildoutDegree = 1;
int maxEntities = 12;
String entityList = "{\"ENTITIES\": [{\"ENTITY_ID\":"+entityID+"}, {\"ENTITY_ID\":"+entityID2+"}]}";

int return_code = g2engine.findNetworkByEntityID(
                            entityList, 
                            maxDegree, 
                            buildoutDegree, 
                            maxEntities,
                            response);
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="2a8797b7-91d9-4be1-a247-ea86de462e9d" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('2a8797b7-91d9-4be1-a247-ea86de462e9d').appendChild(renderjson({"ENTITY_PATHS":[],"ENTITIES":[{"RESOLVED_ENTITY":{"ENTITY_ID":7,"ENTITY_NAME":"Madelyn Caryn","RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":2,"FIRST_SEEN_DT":"2022-11-05 16:18:01.451","LAST_SEEN_DT":"2022-11-05 16:18:13.745"}],"LAST_SEEN_DT":"2022-11-05 16:18:13.745"},"RELATED_ENTITIES":[]}]}))});

#### findNetworkByRecordID

The `findNetworkByRecordID()` function
can be used to find all entities surrounding a requested set of entities.
This includes the requested entities, paths between them, and relations to
other nearby entities.

###### Parameters

- **recordList:** (str) A list of entities, specified by Record ID
  (JSON document)
- **maxDegree:** (int) The maximum number of degrees in paths between search
  entities
- **buildoutDegree:** (int) The number of degrees of relationships to show around
  each search entity
- **maxEntities:** (int) The maximum number of entities to return in the
  discovered network
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later
- **g2EngineFlags:** (long) control flags (applicable flags listed here [findNetworkByRecordID flags](senzing-G2Engine-flags.ipynb#findNetworkByRecordID))

The functions return a JSON document that identifies the path between the
each set of search entities (if the path exists), and the information on the
entities in question (search entities, path entities, and build-out entities.

In [66]:
StringBuffer response = new StringBuffer();
int maxDegree = 2;
int buildoutDegree = 1;
int maxEntities = 12;
String recordList = "{\"RECORDS\": [{\"RECORD_ID\": \""+recordID1+"\", \"DATA_SOURCE\": \""+dataSourceCode1+"\" }, {\"RECORD_ID\": \""+recordID2+"\", \"DATA_SOURCE\": \""+dataSourceCode2+"\" }]}";

int return_code = g2engine.findNetworkByRecordID(
                            recordList, 
                            maxDegree, 
                            buildoutDegree, 
                            maxEntities, 
                            response);
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="6d62286a-68b3-4dc8-a37f-ff2d78554529" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('6d62286a-68b3-4dc8-a37f-ff2d78554529').appendChild(renderjson({"ENTITY_PATHS":[{"START_ENTITY_ID":7,"END_ENTITY_ID":8,"ENTITIES":[]}],"ENTITIES":[{"RESOLVED_ENTITY":{"ENTITY_ID":7,"ENTITY_NAME":"Madelyn Caryn","RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":2,"FIRST_SEEN_DT":"2022-11-05 16:18:01.451","LAST_SEEN_DT":"2022-11-05 16:18:13.745"}],"LAST_SEEN_DT":"2022-11-05 16:18:13.745"},"RELATED_ENTITIES":[]},{"RESOLVED_ENTITY":{"ENTITY_ID":8,"ENTITY_NAME":"Max W Miller","RECORD_SUMMARY":[{"DATA_SOURCE":"TEST","RECORD_COUNT":1,"FIRST_SEEN_DT":"2022-11-05 16:18:13.899","LAST_SEEN_DT":"2022-11-05 16:18:21.309"}],"LAST_SEEN_DT":"2022-11-05 16:18:21.309"},"RELATED_ENTITIES":[]}]}))});

## Connection details

### findInterestingEntities

`findInterestingEntitiesByEntityID` and `findInterestingEntitiesByRecordID` are early adopter features, contact support for their use.

The `whyEntities()`, `whyRecords()`, `whyEntityByRecordID()`,`whyEntityByEntityID()`, and `howEntityByEntityID()` functions can be used
to determine why and how records belong to their resolved entities.
These functions will compare the record data within an entity against the
rest of the entity data, and show why they are connected.
This is calculated based on the features that record data represents.

###### Parameters

- **dataSourceCode:** (str) The data source for the record to be analyzed
- **entityID:** (int) The entity ID for the entity to be analyzed
- **recordID:** (str) The record ID for the record to be analyzed
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later
- **g2EngineFlags:** (long) Control flags

The function returns a JSON document that gives the results of the record
analysis.
The document contains a section called "WHY_RESULTS",
which shows how specific records relate to the rest of the entity.
It has a "WHY_KEY", which is similar to a match key, in defining the relevant
connected data.
It shows candidate keys for features that initially cause the records
to be analyzed for a relationship,
plus a series of feature scores that show how similar the feature data was.

The response document also contains a separate ENTITIES section,
with the full information about the resolved entity.
(Note: When working with this entity data,
Senzing recommends using the flags `G2_ENTITY_OPTION_INCLUDE_INTERNAL_FEATURES`
and `G2_ENTITY_OPTION_INCLUDE_FEATURE_STATS`.
This will provide detailed feature data that is not included by default,
but is useful for understanding the WHY_RESULTS data.)

### whyEntities


###### Parameters

- **entityID1:** (int) The entity ID for the first entity to be analyzed
- **entityID2:** (int) The entity ID for the second entity to be analyzed
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.
- **g2EngineFlags:** (long) control flags (applicable flags listed here [whyEntities flags](senzing-G2Engine-flags.ipynb#whyEntities))


In [67]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.whyEntities(
                            entityID1,
                            entityID2,
                            response);
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);       

<div id="dab7ff00-9f02-4184-92bf-f740027d1102" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('dab7ff00-9f02-4184-92bf-f740027d1102').appendChild(renderjson({"WHY_RESULTS":[{"ENTITY_ID":7,"ENTITY_ID_2":7,"MATCH_INFO":{"WHY_KEY":"+NAME+SSN","WHY_ERRULE_CODE":"CNAME","MATCH_LEVEL_CODE":"NAME_ONLY","CANDIDATE_KEYS":{"NAME_KEY":[{"FEAT_ID":34,"FEAT_DESC":"KRN|MTLN"},{"FEAT_ID":35,"FEAT_DESC":"KRN|MTLN|SSN=1111"}]},"DISCLOSED_RELATIONS":{},"FEATURE_SCORES":{"NAME":[{"INBOUND_FEAT_ID":33,"INBOUND_FEAT":"Madelyn Caryn","INBOUND_FEAT_USAGE_TYPE":"PRIMARY","CANDIDATE_FEAT_ID":33,"CANDIDATE_FEAT":"Madelyn Caryn","CANDIDATE_FEAT_USAGE_TYPE":"PRIMARY","GNR_FN":100,"GNR_SN":100,"GNR_GN":100,"GENERATION_MATCH":-1,"GNR_ON":-1,"SCORE_BUCKET":"SAME","SCORE_BEHAVIOR":"NAME"}],"SSN":[{"INBOUND_FEAT_ID":3,"INBOUND_FEAT":"111-11-1111","INBOUND_FEAT_USAGE_TYPE":"","CANDIDATE_FEAT_ID":3,"CANDIDATE_FEAT":"111-11-1111","CANDIDATE_FEAT_USAGE_TYPE":"","FULL_SCORE":100,"SCORE_BUCKET":"SAME","SCORE_

### whyRecords

###### Parameters

- **dataSourceCode1:** (str) The name of the first data source the record
  is associated with.
  This value is configurable to the system
- **recordID1:** (int) The record ID for the first record to be analyzed
- **dataSourceCode2:** (str) The name of the second data source the record
  is associated with.
  This value is configurable to the system
- **recordID2:** (int) The record ID for the second record to be analyzed
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.
- **g2EngineFlags:** (long) control flags (applicable flags listed here [whyRecords flags](senzing-G2Engine-flags.ipynb#whyRecords))

In [68]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.whyRecords(
                            dataSourceCode1,
                            recordID1,
                            dataSourceCode2,
                            recordID2,
                            response);
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);       

<div id="b06dc21a-afb5-4f84-bae7-2bbe49628818" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('b06dc21a-afb5-4f84-bae7-2bbe49628818').appendChild(renderjson({"WHY_RESULTS":[{"INTERNAL_ID":7,"ENTITY_ID":7,"FOCUS_RECORDS":[{"DATA_SOURCE":"TEST","RECORD_ID":"1"}],"INTERNAL_ID_2":8,"ENTITY_ID_2":8,"FOCUS_RECORDS_2":[{"DATA_SOURCE":"TEST","RECORD_ID":"2"}],"MATCH_INFO":{"WHY_KEY":"","WHY_ERRULE_CODE":"","MATCH_LEVEL_CODE":"","CANDIDATE_KEYS":{},"DISCLOSED_RELATIONS":{},"FEATURE_SCORES":{}}}],"ENTITIES":[{"RESOLVED_ENTITY":{"ENTITY_ID":7,"ENTITY_NAME":"Madelyn Caryn","FEATURES":{"ID_KEY":[{"FEAT_DESC":"SSN=111-11-1111","LIB_FEAT_ID":10,"FEAT_DESC_VALUES":[{"FEAT_DESC":"SSN=111-11-1111","LIB_FEAT_ID":10,"USED_FOR_CAND":"Y","USED_FOR_SCORING":"N","ENTITY_COUNT":8,"CANDIDATE_CAP_REACHED":"Y","SCORING_CAP_REACHED":"N","SUPPRESSED":"N"}]}],"NAME":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33,"USAGE_TYPE":"PRIMARY","FEAT_DESC_VALUES":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33,"

### whyEntityByEntityID

###### Parameters

- **entityID:** (int) The entity ID for the entity to be analyzed
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.
- **g2EngineFlags:** (long) control flags (applicable flags listed here [whyEntityByEntityID flags](senzing-G2Engine-flags.ipynb#whyEntityByEntityID))

In [69]:
StringBuffer response = new StringBuffer();


int return_code = g2engine.whyEntityByEntityID(entityID1,response);


if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="f0c799d6-dce7-4a32-bd45-3558e509e6db" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('f0c799d6-dce7-4a32-bd45-3558e509e6db').appendChild(renderjson({"WHY_RESULTS":[{"INTERNAL_ID":7,"ENTITY_ID":7,"FOCUS_RECORDS":[{"DATA_SOURCE":"TEST","RECORD_ID":"1"},{"DATA_SOURCE":"TEST","RECORD_ID":"8"}],"MATCH_INFO":{"WHY_KEY":"","WHY_ERRULE_CODE":"","MATCH_LEVEL_CODE":"","CANDIDATE_KEYS":{},"FEATURE_SCORES":{}}}],"ENTITIES":[{"RESOLVED_ENTITY":{"ENTITY_ID":7,"ENTITY_NAME":"Madelyn Caryn","FEATURES":{"ID_KEY":[{"FEAT_DESC":"SSN=111-11-1111","LIB_FEAT_ID":10,"FEAT_DESC_VALUES":[{"FEAT_DESC":"SSN=111-11-1111","LIB_FEAT_ID":10,"USED_FOR_CAND":"Y","USED_FOR_SCORING":"N","ENTITY_COUNT":8,"CANDIDATE_CAP_REACHED":"Y","SCORING_CAP_REACHED":"N","SUPPRESSED":"N"}]}],"NAME":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33,"USAGE_TYPE":"PRIMARY","FEAT_DESC_VALUES":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33,"USED_FOR_CAND":"N","USED_FOR_SCORING":"Y","ENTITY_COUNT":1,"CANDIDATE_CAP_REACH

### whyEntityByRecordID

###### Parameters

- **dataSourceCode:** (str) The name of the data source the record
  is associated with.
  This value is configurable to the system
- **recordID:** (int) The record ID for the record to be analyzed
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.
- **g2EngineFlags:** (long) control flags (applicable flags listed here [whyEntityByRecordID flags](senzing-G2Engine-flags.ipynb#whyEntityByRecordID))

In [70]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.whyEntityByRecordID(
                            dataSourceCode1,
                            recordID1,
                            response);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="7730adbc-f544-4455-b6dd-e054def51c9a" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('7730adbc-f544-4455-b6dd-e054def51c9a').appendChild(renderjson({"WHY_RESULTS":[{"INTERNAL_ID":7,"ENTITY_ID":7,"FOCUS_RECORDS":[{"DATA_SOURCE":"TEST","RECORD_ID":"1"},{"DATA_SOURCE":"TEST","RECORD_ID":"8"}],"MATCH_INFO":{"WHY_KEY":"","WHY_ERRULE_CODE":"","MATCH_LEVEL_CODE":"","CANDIDATE_KEYS":{},"FEATURE_SCORES":{}}}],"ENTITIES":[{"RESOLVED_ENTITY":{"ENTITY_ID":7,"ENTITY_NAME":"Madelyn Caryn","FEATURES":{"ID_KEY":[{"FEAT_DESC":"SSN=111-11-1111","LIB_FEAT_ID":10,"FEAT_DESC_VALUES":[{"FEAT_DESC":"SSN=111-11-1111","LIB_FEAT_ID":10,"USED_FOR_CAND":"Y","USED_FOR_SCORING":"N","ENTITY_COUNT":8,"CANDIDATE_CAP_REACHED":"Y","SCORING_CAP_REACHED":"N","SUPPRESSED":"N"}]}],"NAME":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33,"USAGE_TYPE":"PRIMARY","FEAT_DESC_VALUES":[{"FEAT_DESC":"Madelyn Caryn","LIB_FEAT_ID":33,"USED_FOR_CAND":"N","USED_FOR_SCORING":"Y","ENTITY_COUNT":1,"CANDIDATE_CAP_REACH

### howEntityByEntityID

###### Parameters

- **entity_id:** (int) The entity ID for the entity to be analyzed
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later.

Create some records to show resolution steps

In [71]:
String data1 =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Joe\", \"NAME_LAST\": \"Schmoe\", \"SSN_NUMBER\": \"111-22-3456\", \"DOB\": \"03-22-1982\"}";
String data2 =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Joe\", \"NAME_LAST\": \"Schmoe\", \"SSN_NUMBER\": \"111-22-3456\", \"CITIZENSHIP\": \"British\"}";
String data3 =  "{\"NAME_TYPE\": \"PRIMARY\", \"NAME_FIRST\": \"Joe\", \"NAME_LAST\": \"Schmoe\", \"SSN_NUMBER\": \"111-22-3456\", \"NATIONALITY\": \"Australian\"}";

String recordID1 = "10";
String recordID2 = "11";
String recordID3 = "12";
String dataSourceCode = "TEST";

g2engine.addRecord(dataSourceCode, recordID1, data1, loadID);
g2engine.addRecord(dataSourceCode, recordID2, data2, loadID);
g2engine.addRecord(dataSourceCode, recordID3, data3, loadID);

StringBuffer response = new StringBuffer();

g2engine.getEntityByRecordID(dataSourceCode, recordID1, response);
JsonObject jsonObject = JsonUtil.parseJsonObject(response.toString());
long entityID = jsonObject.getJsonObject("RESOLVED_ENTITY").getJsonNumber("ENTITY_ID").longValue();

In [72]:
StringBuffer response = new StringBuffer();

int return_code = g2engine.howEntityByEntityID(
                            entityID,
                            response);
                            
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    RenderJSON(response);

<div id="1005ae71-751e-4998-85d5-f1554abc1e0f" style="height:100%; width:100%; background-color: LightCyan"></div>

require(["https://rawgit.com/caldwell/renderjson/master/renderjson.js"], function() {document.getElementById('1005ae71-751e-4998-85d5-f1554abc1e0f').appendChild(renderjson({"HOW_RESULTS":{"RESOLUTION_STEPS":[{"STEP":1,"VIRTUAL_ENTITY_1":{"VIRTUAL_ENTITY_ID":"V11","MEMBER_RECORDS":[{"INTERNAL_ID":11,"RECORDS":[{"DATA_SOURCE":"TEST","RECORD_ID":"11"}]}]},"VIRTUAL_ENTITY_2":{"VIRTUAL_ENTITY_ID":"V12","MEMBER_RECORDS":[{"INTERNAL_ID":12,"RECORDS":[{"DATA_SOURCE":"TEST","RECORD_ID":"12"}]}]},"INBOUND_VIRTUAL_ENTITY_ID":"V12","RESULT_VIRTUAL_ENTITY_ID":"V11-S1","MATCH_INFO":{"MATCH_KEY":"+NAME+SSN","ERRULE_CODE":"SF1_PNAME_CSTAB","FEATURE_SCORES":{"NAME":[{"INBOUND_FEAT_ID":46,"INBOUND_FEAT":"Joe Schmoe","INBOUND_FEAT_USAGE_TYPE":"PRIMARY","CANDIDATE_FEAT_ID":46,"CANDIDATE_FEAT":"Joe Schmoe","CANDIDATE_FEAT_USAGE_TYPE":"PRIMARY","GNR_FN":100,"GNR_SN":100,"GNR_GN":100,"GENERATION_MATCH":-1,"GNR_ON":-1,"SCORE_BUCKET":"SAME","SCORE_BEHAVIOR":"NAME"}],"SSN":[{"INBOUND_FEAT_ID":47,"INBOUND_FEAT":

## Reporting

Exporting entity data from resolved entities is one of the core purposes of
Senzing software.
In just a few short steps, the Senzing engine allows users to export entity
data in either JSON or CSV format.

### exportJSONEntityReport

There are three steps to exporting resolved entity data from the G2Engine
object in JSON format.
First, use the `exportJSONEntityReport()` method to generate the `exportHandle`.

###### Parameters

- **exportHandle:** (long) A long from which resolved entity data may be
- **g2EngineFlags:** (long) control flags (applicable flags listed here [exportJSONEntityReport flags](senzing-G2Engine-flags.ipynb#exportJSONEntityReport))

In [73]:
StringBuffer response = new StringBuffer();
Result<Long> exportHandle = new Result<Long>();

long g2EngineFlags = g2engine.G2_EXPORT_DEFAULT_FLAGS;
int return_code = g2engine.exportJSONEntityReport(g2EngineFlags, exportHandle);

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code);

0

### fetchNext

Second, use the `fetchNext()` method to read the exportHandle and export a
row of JSON output containing the entity data for a single entity.
Note that successive calls of `fetchNext()` will export successive rows of
entity data.

###### Parameters

- **exportHandle:** (long) A long from which resolved entity data may be
  read and exported.
- **response:** (StringBuffer) Object to store the output of the method. Can be used to print or store for later

In [74]:
while(true){
    StringBuffer response = new StringBuffer();
    int return_code = g2engine.fetchNext(exportHandle.getValue(), response);
    if(return_code != 0){
        System.out.print(g2engine.getLastException());
        break;
    }
    if(response.length() == 0){
        break;
    }
    System.out.println(response.toString());
}

{"RESOLVED_ENTITY":{"ENTITY_ID":1,"ENTITY_NAME":"JANE SMITH","FEATURES":{"ADDRESS":[{"FEAT_DESC":"653 STATE ROUTE 7 FRESNO CA 55073-1234","LIB_FEAT_ID":2,"USAGE_TYPE":"HOME","FEAT_DESC_VALUES":[{"FEAT_DESC":"653 STATE ROUTE 7 FRESNO CA 55073-1234","LIB_FEAT_ID":2}]}],"NAME":[{"FEAT_DESC":"JANE SMITH","LIB_FEAT_ID":1,"USAGE_TYPE":"PRIMARY","FEAT_DESC_VALUES":[{"FEAT_DESC":"JANE SMITH","LIB_FEAT_ID":1}]}],"SSN":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3,"FEAT_DESC_VALUES":[{"FEAT_DESC":"111-11-1111","LIB_FEAT_ID":3}]}]},"RECORDS":[{"DATA_SOURCE":"TEST","RECORD_ID":"06C29624478FF1C3285E294E5657796E3A67244A","ENTITY_TYPE":"GENERIC","INTERNAL_ID":1,"ENTITY_KEY":"F0932495C523ED54B7E0B8D326F82AA6F4F3FE0D","ENTITY_DESC":"JANE SMITH","MATCH_KEY":"","MATCH_LEVEL":0,"MATCH_LEVEL_CODE":"","ERRULE_CODE":"","LAST_SEEN_DT":"2022-11-05 16:18:07.592"}]},"RELATED_ENTITIES":[]}

{"RESOLVED_ENTITY":{"ENTITY_ID":3,"ENTITY_NAME":"Lily Owens","FEATURES":{"NAME":[{"FEAT_DESC":"Lily Owens","LIB_FEAT_ID":15,"U

### closeExport

###### Parameters

- **exportHandle:** (long) A long from which resolved entity data may be
  read and exported.

In [75]:
int return_code = g2engine.closeExport(exportHandle.getValue());

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code);

0

### exportCSVEntityReport

There are three steps to exporting resolved entity data from the G2Engine
object in CSV format.
First, use the `exportCSVEntityReport()` method to generate a long integer,
referred to here as an 'exportHandle'.

###### Parameters

- **headers:** (str) A comma-separated list of column names for the CSV
  export. (These are listed a little further down.)
- **g2EngineFlags:** (long) control flags (applicable flags listed here [exportCSVEntityReport flags](senzing-G2Engine-flags.ipynb#exportCSVEntityReport))
- **exportHandle:** (long) A long from which resolved entity data may be
  read and exported.
  
Second, use the `fetchNext()` method to read the exportHandle and export a
row of CSV output containing the entity data for a single entity.
Note that the first call of `fetchNext()` will yield a header row,
and that successive calls of `fetchNext()` will export successive rows of
entity data.

In [76]:
String headers = "RESOLVED_ENTITY_ID,RESOLVED_ENTITY_NAME,RELATED_ENTITY_ID,MATCH_LEVEL,MATCH_KEY,IS_DISCLOSED,IS_AMBIGUOUS,DATA_SOURCE,RECORD_ID,JSON_DATA,LAST_SEEN_DT,NAME_DATA,ATTRIBUTE_DATA,IDENTIFIER_DATA,ADDRESS_DATA,PHONE_DATA,RELATIONSHIP_DATA,ENTITY_DATA,OTHER_DATA";
Result<Long> exportHandle = new Result<Long>();
StringBuffer response = new StringBuffer();

g2engine.exportCSVEntityReport(headers, g2EngineFlags, exportHandle);

while(true){
    StringBuffer response = new StringBuffer();
    int return_code = g2engine.fetchNext(exportHandle.getValue(), response);
    if(return_code != 0){
        System.out.print(g2engine.getLastException());
        break;
    }
    if(response.length() == 0){
        break;
    }
    System.out.println(response.toString());
}
g2engine.closeExport(exportHandle.getValue());

RESOLVED_ENTITY_ID,RESOLVED_ENTITY_NAME,RELATED_ENTITY_ID,MATCH_LEVEL,MATCH_KEY,IS_DISCLOSED,IS_AMBIGUOUS,DATA_SOURCE,RECORD_ID,JSON_DATA,LAST_SEEN_DT,NAME_DATA,ATTRIBUTE_DATA,IDENTIFIER_DATA,ADDRESS_DATA,PHONE_DATA,RELATIONSHIP_DATA,ENTITY_DATA,OTHER_DATA

1,"JANE SMITH",0,0,"",0,0,"TEST","06C29624478FF1C3285E294E5657796E3A67244A","{""NAME_TYPE"":""PRIMARY"",""NAME_FIRST"":""JANE"",""NAME_LAST"":""SMITH"",""ADDR_TYPE"":""HOME"",""ADDR_LINE1"":""653 STATE ROUTE 7"",""ADDR_CITY"":""FRESNO"",""ADDR_STATE"":""CA"",""ADDR_POSTAL_CODE"":""55073-1234"",""SSN_NUMBER"":""111-11-1111"",""DATA_SOURCE"":""TEST"",""ENTITY_TYPE"":""GENERIC"",""DSRC_ACTION"":""A"",""RECORD_ID"":""""}","2022-11-05 16:18:07.592","PRIMARY: SMITH JANE","","SSN: 111-11-1111","HOME: 653 STATE ROUTE 7 FRESNO CA 55073-1234","","","",""

3,"Lily Owens",0,0,"",0,0,"TEST","4","{""NAME_TYPE"":""PRIMARY"",""NAME_FIRST"":""Lily"",""NAME_LAST"":""Owens"",""SSN_NUMBER"":""111-11-1111"",""DATA_SOURCE"":""TEST"",""ENTITY_TYPE"":""GEN

0

## Cleanup

### purgeRepository

**Before** calling `purgeRepository()` all other instances of the Senzing API (whether in custom code, REST API, stream-loader, redoer, G2Loader, etc) **MUST** be destroyed or shutdown.

In [77]:
int return_code= g2engine.purgeRepository();
if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code)

0

### destroy
Once all searching is done in a process call `destroy()` to uninitialize Senzing and clean up resources. You should always do this once at the end of each process.

In [78]:
int return_code = g2engine.destroy();

if(return_code!=0)
    System.out.print(g2engine.getLastException());
else
    System.out.print(return_code);

0